In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("data"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1)

# Any results you write to the current directory are saved as output.

2024-05-28 23:31:03.415082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 23:31:03.415124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 23:31:03.416293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-28 23:31:03.422584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 23:31:04.086385: W tensorflow/compiler/tf2

['Hindi_English_Truncated_Corpus.csv', 'images_of_mammals']


In [2]:
lines=pd.read_csv("data/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [3]:
lines['source'].value_counts()

source
tides        50000
ted          39881
indic2012    37726
Name: count, dtype: int64

In [4]:
#lines=lines[lines['source']=='ted']

In [5]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,indic2012,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,"The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .","कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी ."
6,indic2012,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
9,tides,You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .,"हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . ."


In [6]:
pd.isnull(lines).sum()

source              0
english_sentence    2
hindi_sentence      0
dtype: int64

In [7]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [8]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [9]:
lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 3)

In [10]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [11]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [12]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [13]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [14]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
lines.head()

,source,english_sentence,hindi_sentence
25520,indic2012,islam is word from arabic and it full word is salamaits definition peace surrender,START_ इस्लाम शब्द अरबी भाषा का शब्द है जिसका मूल शब्द सल्लमा है जिस की दो परिभाषाएं हैं शान्ति आत्मसमर्पण। _END
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END
111804,tides,the provision would not affect the power of parliament to make laws in respect of income from professions etc lrb article rrb,START_ व्यवसायों आदि से होने वाली आय के बारे में विधि बनाने की संसद की शक्ति पर उपबंध का प्रभाव नहीं पड़ेगा अनुच्छेद _END


In [16]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [17]:
len(all_eng_words)

30899

In [18]:
all_eng_words

{'shilpagram',
 'satisfactorily',
 'rajjukas',
 'congressmen',
 'shepherd',
 'antaryamiri',
 'warm',
 'gem',
 'nineteenth',
 'kaikaees',
 'betwa',
 'dominants',
 'protocols',
 'awesome',
 'saini',
 'conventions',
 'preparation',
 'virendernagar',
 'bharavsorya',
 'forgiven',
 'lawkarma',
 'learn',
 'bedrock',
 'plural',
 'basic',
 'uttranchal',
 'muhamed',
 'rollercoaster',
 'monkey',
 'duly',
 'spectacle',
 'plenty',
 'triangular',
 'noc',
 'congressled',
 'idai',
 'hindiblogsorg',
 'microbiologists',
 'unification',
 'heartfelt',
 'desaturated',
 'cordial',
 'sixthseventh',
 'coastal',
 'cycle',
 'jahangir',
 'mood',
 'chitrangadha',
 'complitation',
 'specialities',
 'mint',
 'sastrita',
 'participation',
 'angles',
 'wateroutlet',
 'workand',
 'underlying',
 'sevenfold',
 'indifference',
 'simplifying',
 'moviemakers',
 'prohibition',
 'undisturbed',
 'watchman',
 'nalini',
 'preachers',
 'january',
 'modellers',
 'professionally',
 'dangling',
 'painful',
 'odyssi',
 'naveen',
 'a

In [19]:
len(all_hindi_words)

36900

In [20]:
all_hindi_words

{'बंगला',
 'पडे़',
 'बरले',
 'लिकाऋ',
 'घुलमिल',
 'बीजीएमएल',
 'शिवलीलापरक',
 'पृष्ठभुमि',
 'शाहजहां',
 'लिखिए',
 'लगवाये',
 'समझबूझ',
 'रखेंगेराज्य',
 'हैपीड़ितों',
 'कैविटी',
 'पडनेवाले',
 'हरेभरे',
 'बाँटा',
 'कऋऊण्श्छ्ष्या',
 'किलं',
 'करानी',
 'प्लेटफ़ॉर्म',
 'उपन्यायाधीश',
 'conventions',
 'जमईका',
 'थायाज़ाइड',
 'जांघिया',
 '“गडबड',
 'वस्त्रों',
 'नियुक्तः',
 'सीईडीवी',
 'डायटमों',
 'मृदु',
 'जांघों',
 'किंवदन्तियाँ',
 'रेम्ब्रान्ट',
 'मार्गदर्शिका',
 'संचिका',
 'पराधीन',
 'थीकि',
 'छेडऋते',
 'कंअं',
 'ताम्रकांस्यकालीन',
 'ढिऋर',
 'गहरायी',
 'तनावपूर्ण',
 'असमर्थनीय',
 'पदयात्रियों',
 'ग्लूकोज',
 'केय',
 'उपनगरों',
 'उठा',
 'सुबाहु',
 'पुर्ण',
 'देशोके',
 'फ़ीट',
 'पेशियों',
 'मुल्तानी',
 'cycle',
 'संकीर्ण',
 'खिलाडियों',
 'अनुभवी',
 'शोरगुल',
 'रूपी',
 'न्न्गौरी',
 'मुहल्ले',
 'नियुक़्त',
 'मानसअयोध्या',
 'विधिंयों',
 'ओझुकऋऊण्श्छ्ष्कम',
 'घरेलू',
 'स्वामिभक़्ति',
 'फ्लोडाराग्राम',
 'डिनर',
 'वेंकटेश्वर',
 'त्रितलीय',
 'आवश्यक्तानुसार',
 'उत्पादन',
 'बहुतम',
 'बुलई',
 'सचेतन',

In [21]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [22]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
25520,indic2012,islam is word from arabic and it full word is salamaits definition peace surrender,START_ इस्लाम शब्द अरबी भाषा का शब्द है जिसका मूल शब्द सल्लमा है जिस की दो परिभाषाएं हैं शान्ति आत्मसमर्पण। _END,14,21
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END,7,9
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END,6,9
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END,4,6
111804,tides,the provision would not affect the power of parliament to make laws in respect of income from professions etc lrb article rrb,START_ व्यवसायों आदि से होने वाली आय के बारे में विधि बनाने की संसद की शक्ति पर उपबंध का प्रभाव नहीं पड़ेगा अनुच्छेद _END,22,24


In [23]:
lines[lines['length_eng_sentence']>30].shape

(2463, 5)

In [24]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [25]:
lines

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
118633,ted,everything is reliant on these computers working,START_ इन कंप्यूटरों पर सब कुछ निर्भर है _END,7,9
113495,tides,parliament does not control the government,START_ संसद का सरकार पपर नियंत्रण नपहीं रहता _END,6,9
29783,tides,race equality new laws,START_ नये कानून नस्ली समानता _END,4,6
57202,ted,there was lasagna there was casseroles,START_ वहां लाजान्या था कैसेरोल थे _END,6,7
107821,indic2012,super power india source google writer vedpratap vedik,START_ महाशक्ति भारत गूगल पुस्तक लेखक वेदप्रताप वैदिक _END,8,9
...,...,...,...,...,...
71827,tides,natural waters contain only a few parts per billion of mercury,START_ प्राकृतिक जल में तो एक अरब भाग में पारे के केवल कुछ ही अंश होते हैं _END,11,18
27219,tides,the association was not called upon to resort to regulation of output during the next three decades,START_ अगले तीन दशकों में उत्पादन के नियमन के लिए ऐसोसियेशन के पास जाने की आवश्यकता नहीं पड़ी _END,17,19
101602,indic2012,pandit motilal nehru was chairman of this committee and subhashbabu was a member,START_ पंडित मोतीलाल नेहरू इस आयोग के अध्यक्ष और सुभाषबाबू उसके एक सदस्य थे। _END,13,15
2505,ted,which is a way of bringing water directly to the plant stock,START_ जो कि पानी को सीधे पौधे की जड में पहुँचाने का एक तरीका है। _END,12,16


In [26]:
lines.shape

(16512, 5)

In [27]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [28]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [29]:
print(max_length_src)

20


In [30]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(30899, 36900)

In [31]:
num_decoder_tokens += 1 #for zero padding


In [32]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [33]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [34]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
27151,ted,that may or may not work out,START_ जो हो सकता है कामयाब हो या न हो। _END,7,11
38213,indic2012,picture large lesson,START_ चित्रमोटा पाठ _END,3,4
84397,tides,some important principles have however been laid down in the karachi resolution and in the election manifesto,START_ फिर भी कराची कांग्रेस के प्रस्ताव और चुनाव घोषणपत्र में कुछ अहम सिद्धांत तय कर दिये गये हैं _END,17,20
27258,ted,now this lady she used to do the winding of the yarn,START_ और ये स्त्री जो कि कताई का काम करती थी _END,12,12
41559,indic2012,most of the words of all indian languages are either directly borrowed or derived from sanskrit,START_ इनकी अधिकांश शब्दावली या तो संस्कृत से ली गयी है या संस्कृत से प्रभावित है। _END,16,17
104979,indic2012,he was awarded “”novel king“” during his lifetime,START_ उन्हें अपने जीवन काल में ही ‘उपन्यास सम्राट की पदवी मिल गयी थी। _END,8,15
37155,ted,youre allowed to just hate them,START_ आपको पूरी छुट है उनसे नफरत करनेकी _END,6,9
71793,tides,” with trembling heart i stood before you in dread of your ominous frown,START_ कांपते हृदय से मैं तुम्हारे सम्मुख खड़ा था तुम्हारी भृकुटि से भयभीत _END,14,14
15299,ted,remember every spot on here is a galaxy and you see the galaxies,START_ ध्यान दें यहाँ हर एक बिन्दु एक आकाशगंगा है आप उन आकाशगंगाओं को भी देख पा रहे हैं _END,13,20
16289,indic2012,harivanshray bachchan,START_ हरिवंश राय बच्चन _END,2,5


### Split the data into train and test

In [35]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((13209,), (3303,))

In [36]:
X_train

46191     general musharraf needs a dialogue with india                     
86767     the way is against the gobhakshan and muslim culture              
69619     a device to change the roman unicode to sanskrit                  
19368     are basically as dumb as dumps                                    
10100     the bureaucrat news in hindi                                      
                      ...                                                   
49341     this excavation in the prerashtrakuta series can be dated near ad 
37687     we need to help our physicians and our nurses                     
109980    i was just three years old when my brother came along             
121169    we had a whole choice of inappropriate names for her              
19859     and our ability to connect with each other regardless of geography
Name: english_sentence, Length: 13209, dtype: object

In [37]:
y_train

46191     START_ जनरल मुशर्रफ को भी भारत से बातचीत करने की जरूरत है _END                           
86767     START_ इनके पंथ मुसलमानसंस्कृति और गोभक्षण के विरोधी थे। _END                            
69619     START_ रोमन को यूनिकोड संस्कृत में लिप्यंतरित करने का उपकरण _END                         
19368     START_ बहुत ज्यादा बेवकूफ़ जीव होते हैं। _END                                            
10100     START_ द ब्युरोक्रैट न्यूज हिन्दी _END                                                   
                           ...                                                                     
49341     START_ राष्ट्रकूट पूर्व श्रृंखलामें इस उत्खनन का समय ईसवी सन् के आसपास का हो सकता है _END
37687     START_ हमें चिकित्सकों और नर्सों की मदद करनी है _END                                     
109980    START_ मैं सिर्फ़ साल की थी जब मेरा भाई इस दुनिया में आया _END                           
121169    START_ हम उसे कई किस्मों के अनुचित नामों से पुकारा करते थे _END                          


### Let us save this data

In [38]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [39]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [40]:
latent_dim=300

In [41]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

2024-05-28 23:31:06.724082: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 23:31:06.778426: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 23:31:06.778699: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [42]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [43]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [44]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 300)            9269700   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            1107030   ['input_2[0][0]']             
                                                          0                                   

In [45]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 100

In [46]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



/tmp/ipykernel_123398/2713035456.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/100


2024-05-28 23:31:12.007039: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_36/output/_23'
2024-05-28 23:31:12.458529: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-05-28 23:31:12.719803: I external/local_xla/xla/service/service.cc:168] XLA service 0x7304f4ad2c50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-28 23:31

206/206 [==============================] - 63s 285ms/step - loss: 7.5422 - val_loss: 7.1151
Epoch 2/100
206/206 [==============================] - 53s 259ms/step - loss: 6.9663 - val_loss: 7.0215
Epoch 3/100
206/206 [==============================] - 53s 260ms/step - loss: 6.9121 - val_loss: 7.0095
Epoch 4/100
206/206 [==============================] - ETA: 0s - loss: 6.8833

KeyboardInterrupt: 

In [ ]:
model.save_weights('nmt_weights.h5')


In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
model.save('models/english_to_hindi_translator.h5')